### Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import os
from masserstein import NMRSpectrum, estimate_proportions
import matplotlib.pyplot as plt
import pulp
import pickle
import itertools
import matplotlib.pyplot as plt

### Data

In [ ]:
# What to change for PMG 287 whole mixture:

#optionally add ylim(0,0.4) in plots

In [ ]:
experiment_name = 'PMG 287 monitoring with full mixture spectrum'

In [ ]:
mixture_paths = {'Saccharose hydrolysis': '../data/saccharose_hydrolysis/preprocessed_mixture.csv',
                'PMG 284 monitoring': '../data/PMG_284/preprocessed_mixture.csv',
                'PMG 287 monitoring': '../data/PMG_287/preprocessed_mixture_cut_to_regions.csv',
                'Saccharose hydrolysis with full mixture spectrum': '../data/saccharose_hydrolysis_full_mixture_spectrum/preprocessed_mixture.csv',
                'PMG 284 monitoring with full mixture spectrum': '../data/PMG_284_full_mixture_spectrum/preprocessed_mixture_whole.csv',
                'PMG 287 monitoring with full mixture spectrum': '../data/PMG_287_full_mixture_spectrum/preprocessed_mixture_whole.csv'
                }

sucrose_time_axis_path = '../data/saccharose_hydrolysis/time_axis_sucrose_hydrolysis.txt'

reagents_paths = {'Saccharose hydrolysis': ['../data/saccharose_hydrolysis/preprocessed_saccharose.csv',
                                            '../data/saccharose_hydrolysis/preprocessed_alpha_glucose.csv',
                                            '../data/saccharose_hydrolysis/preprocessed_beta_glucose.csv',
                                            '../data/saccharose_hydrolysis/preprocessed_fructose.csv'],
                'PMG 284 monitoring': ['../data/PMG_284/preprocessed_pentene.csv',
                                      '../data/PMG_284/preprocessed_triethylsilane.csv',
                                      '../data/PMG_284/preprocessed_product.csv'],
                'PMG 287 monitoring': ['../data/PMG_287/preprocessed_hexene.csv',
                                       '../data/PMG_287/preprocessed_triethylsilane.csv',
                                       '../data/PMG_287/preprocessed_product.csv'],
                'Saccharose hydrolysis with full mixture spectrum':
                                      ['../data/saccharose_hydrolysis_full_mixture_spectrum/preprocessed_saccharose.csv',
                                        '../data/saccharose_hydrolysis_full_mixture_spectrum/preprocessed_alpha_glucose.csv',
                                       '../data/saccharose_hydrolysis_full_mixture_spectrum/preprocessed_beta_glucose.csv',
                                       '../data/saccharose_hydrolysis_full_mixture_spectrum/preprocessed_fructose.csv'],
                'PMG 284 monitoring with full mixture spectrum': ['../data/PMG_284_full_mixture_spectrum/preprocessed_pentene.csv',
                                                               '../data/PMG_284_full_mixture_spectrum/preprocessed_triethylsilane.csv',
                                                               '../data/PMG_284_full_mixture_spectrum/preprocessed_product.csv'],
                'PMG 287 monitoring with full mixture spectrum': ['../data/PMG_287_full_mixture_spectrum/preprocessed_hexene.csv',
                                                               '../data/PMG_287_full_mixture_spectrum/preprocessed_triethylsilane.csv',
                                                               '../data/PMG_287_full_mixture_spectrum/preprocessed_product.csv'],
                 }

mixture_separators = {'Saccharose hydrolysis': '\t',
                'PMG 284 monitoring': '\t',
                'PMG 287 monitoring': '\t',
                'Saccharose hydrolysis with full mixture spectrum': '\t',
                'PMG 284 monitoring with full mixture spectrum': '\t',
                'PMG 287 monitoring with full mixture spectrum': '\t'
                }

reagents_separators = {'Saccharose hydrolysis': '\t',
                 'PMG 284 monitoring': '\t',
                'PMG 287 monitoring': '\t',
                'Saccharose hydrolysis with full mixture spectrum': '\t',
                'PMG 284 monitoring with full mixture spectrum': '\t',
                'PMG 287 monitoring with full mixture spectrum': '\t'
                }

results_paths = {
                'Saccharose hydrolysis': '../results/saccharose_hydrolysis/',
                'PMG 284 monitoring': '../results/PMG_284/',
                'PMG 287 monitoring': '../results/PMG_287/',
                'Saccharose hydrolysis with full mixture spectrum': '../results/saccharose_hydrolysis_full_mixture_spectrum/',
                'PMG 284 monitoring with full mixture spectrum': '../results/PMG_284_full_mixture_spectrum/',
                'PMG 287 monitoring with full mixture spectrum': '../results/PMG_287_full_mixture_spectrum/'
                }

substances_names = {'Saccharose hydrolysis': ['sucrose', 'alpha-glucose', 'beta-glucose', 'fructose'],
                    'PMG 284 monitoring': ['2-pentene', 'triethylsilane', '2-pentyl-1-triethylsilane'],
                    'PMG 287 monitoring': ['1-hexene', 'triethylsilane', 'hexyl-triethylsilane'],
                    'Saccharose hydrolysis with full mixture spectrum': ['sucrose', 'alpha-glucose', 'beta-glucose', 'fructose'],
                    'PMG 284 monitoring with full mixture spectrum': ['2-pentene', 'triethylsilane', '2-pentyl-1-triethylsilane'],
                    'PMG 287 monitoring with full mixture spectrum': ['1-hexene', 'triethylsilane', 'hexyl-triethylsilane'],
                    }

integrals_to_compare_with = {'Saccharose hydrolysis': '../mnova_integrals/sacharoza_calki_nowe.csv',
                    'PMG 284 monitoring': '../mnova_integrals/284_edited_corrected_calki.csv',
                    'PMG 287 monitoring': '../mnova_integrals/287_calki_zsumowane.csv',
                    'Saccharose hydrolysis with full mixture spectrum': '../mnova_integrals/sacharoza_calki_nowe.csv',
                    'PMG 284 monitoring with full mixture spectrum': '../mnova_integrals/284_edited_corrected_calki.csv',
                    'PMG 287 monitoring with full mixture spectrum': '../mnova_integrals/287_calki_zsumowane.csv'
                            }

integrals_separators = {'Saccharose hydrolysis': '\t',
                    'PMG 284 monitoring': ',',
                    'PMG 287 monitoring': ';',
                    'Saccharose hydrolysis with full mixture spectrum': '\t',
                    'PMG 284 monitoring with full mixture spectrum': ',',
                    'PMG 287 monitoring with full mixture spectrum': ';'
                            }

python_integrals_to_compare_with = {'Saccharose hydrolysis': '../python_integrals/python_integral_Saccharose_hydrolysis.csv',
                    'PMG 284 monitoring': '../python_integrals/python_integral_PMG_284_monitoring.csv',
                    'PMG 287 monitoring': '../python_integrals/python_integral_PMG_287_monitoring.csv',
                    'Saccharose hydrolysis with full mixture spectrum': '../python_integrals/python_integral_Saccharose_hydrolysis.csv',
                    'PMG 284 monitoring with full mixture spectrum': '../python_integrals/python_integral_PMG_284_monitoring.csv',
                    'PMG 287 monitoring with full mixture spectrum': '../python_integrals/python_integral_PMG_287_monitoring.csv'
                                    }

python_integrals_separators = {'Saccharose hydrolysis': ',',
                    'PMG 284 monitoring': ',',
                    'PMG 287 monitoring': ',',
                    'Saccharose hydrolysis with full mixture spectrum': ',',
                    'PMG 284 monitoring with full mixture spectrum': ',',
                    'PMG 287 monitoring with full mixture spectrum': ','
                            }

colors_for_components = {'Saccharose hydrolysis': ['#f5cacc', '#e2eac4', '#c0f5ce', '#c4f1ef'],
                         'PMG 284 monitoring': ['#f5cacc', '#e2eac4', '#c0f5ce'],
                        'PMG 287 monitoring': ['#decdee', '#eed5cd', '#cdeeee'],
                         'Saccharose hydrolysis with full mixture spectrum': ['#f5cacc', '#e2eac4', '#c0f5ce', '#c4f1ef'],
                         'PMG 284 monitoring with full mixture spectrum': ['#f5cacc', '#e2eac4', '#c0f5ce'],
                         'PMG 287 monitoring with full mixture spectrum': ['#decdee', '#eed5cd', '#cdeeee']
                        }

saturated_colors_for_components = {'Saccharose hydrolysis': ['#e07676', '#b3ba97', '#88c398', '#2bbcb7'],
                         'PMG 284 monitoring': ['#e07676', '#b3ba97', '#88c398'],
                        'PMG 287 monitoring': ['#a06fce', '#cb8067', '#67cbcb'],
                        'Saccharose hydrolysis with full mixture spectrum': ['#e07676', '#b3ba97', '#88c398', '#2bbcb7'],
                         'PMG 284 monitoring with full mixture spectrum': ['#e07676', '#b3ba97', '#88c398'],
                        'PMG 287 monitoring with full mixture spectrum': ['#a06fce', '#cb8067', '#67cbcb']
                        }

### Finding all values of kappa, kappa_th

In [ ]:
def convert(txt):
    k = float(txt)
    if k%1 ==0:
        return int(k)
    return k

In [ ]:
all_kappas = []
all_kappas_th = []
for file in os.listdir(results_paths[experiment_name]):
    if file.startswith('noise_proportions_in_times'):
        kappa = convert(file.split('_')[4])
        all_kappas.append(kappa)
        kappa_th = convert(file.split('_')[5][:-4])
        all_kappas_th.append(kappa_th)
        
all_kappas = sorted(list(set(all_kappas)))
all_kappas_th = sorted(list(set(all_kappas_th)))

In [ ]:
kappas_to_remove = [1.2, 0.75]
kappas_th_to_remove = [1.2, 0.75]

if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
    kappas_to_remove = [0.05, 0.5, 0.75, 1, 1.2]
    kappas_th_to_remove = [0.05, 0.5, 0.75, 1, 1.2]

if experiment_name == 'PMG 284 monitoring with full mixture spectrum':
    kappas_to_remove = [0.5]
    kappas_th_to_remove = [0.5]

In [ ]:
all_kappas = sorted(list(set(all_kappas) - set(kappas_to_remove)))
all_kappas_th = sorted(list(set(all_kappas_th) - set(kappas_th_to_remove)))


### Reading results for different values of kappa, kappa_th

In [ ]:
results_dict = {}

for kappa in all_kappas:
    for kappa_th in all_kappas_th:
        try:
            with open(results_paths[experiment_name] + 'proportions_in_times_' +
                                  str(kappa)+'_'+str(kappa_th)+'.pkl', 'rb') as f:
                y = pickle.load(f)
                y = np.array(y)
                results_dict[(kappa, kappa_th)] = y
        except FileNotFoundError:
            results_dict[(kappa, kappa_th)] = None

### Loading integrals computed in MNova

In [ ]:
integrals = pd.read_csv(integrals_to_compare_with[experiment_name], sep=integrals_separators[experiment_name])

if experiment_name == 'PMG 287 monitoring with full mixture spectrum':
    how_to_divide = 'whole spectrum'
else:
    how_to_divide = 'sum'

if experiment_name == 'Saccharose hydrolysis' or experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
    integrals = integrals.iloc[1:,:]
    integrals.columns = ['saccharose', 'alpha-glucose', 'beta-glucose', 'fructose']
    integrals = integrals.apply(pd.to_numeric)
    nominator = np.array(integrals[['saccharose', 'alpha-glucose', 'beta-glucose', 'fructose']])
    if how_to_divide == 'whole spectrum':
        denominator = np.array(integrals[['whole spectrum']])
    elif how_to_divide == 'sum':
        denominator = np.array(integrals[['saccharose', 'alpha-glucose', 
                                          'beta-glucose', 'fructose']]).sum(axis=1).reshape(-1,1)
    mnova_integrals_proportions = nominator/denominator
    
if experiment_name == 'PMG 284 monitoring' or experiment_name == 'PMG 284 monitoring with full mixture spectrum':
    integrals = integrals.iloc[1:,[2,6,8,10]]
    integrals.rename(columns={'Y(X)': 'penten',
                             'Y2(X)': 'silan',
                             'Y3(X)': 'produkt',
                             'Unnamed: 10': 'cale widmo'}, inplace=True)
    integrals = integrals[['penten', 'silan', 'produkt', 'cale widmo']]
    integrals = integrals.apply(pd.to_numeric)
    nominator = np.array(integrals[['penten', 'silan', 'produkt']])
    if how_to_divide == 'whole spectrum':
        denominator = np.array(integrals[['cale widmo']])
    elif how_to_divide == 'sum':
        denominator = np.array(integrals[['penten', 'silan', 'produkt']]).sum(axis=1).reshape(-1,1)
    mnova_integrals_proportions = nominator/denominator
    
if experiment_name == 'PMG 287 monitoring' or experiment_name == 'PMG 287 monitoring with full mixture spectrum':
    integrals = integrals[['hexene sum', 'silane sum', 'product sum', 'all']].iloc[1:]
    for col in integrals.columns:
        integrals[col] = integrals[col].apply(lambda x: float(x.replace(',', '.')))
        
    nominator = np.array(integrals[['hexene sum', 'silane sum', 'product sum']])
    if how_to_divide == 'whole spectrum':
        denominator = np.array(integrals[['all']])
    elif how_to_divide == 'sum':
        denominator = np.array(integrals[['hexene sum', 'silane sum', 'product sum']]).sum(axis=1).reshape(-1,1)
    mnova_integrals_proportions = nominator/denominator

### Loading integrals computed in Python

In [ ]:
integrals = pd.read_csv(python_integrals_to_compare_with[experiment_name], 
                        sep=python_integrals_separators[experiment_name])

if experiment_name == 'PMG 287 monitoring with full mixture spectrum':
    how_to_divide = 'whole spectrum'
else:
    how_to_divide = 'sum'

if experiment_name == 'Saccharose hydrolysis' or experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
    integrals = integrals.apply(pd.to_numeric)
    nominator = np.array(integrals.iloc[:,:4])
    if how_to_divide == 'whole spectrum':
        denominator = np.array(integrals[['whole_spectrum']])
    elif how_to_divide == 'sum':
        denominator = np.array(integrals.iloc[:,:4].sum(axis=1)).reshape(-1,1)
    python_integrals_proportions = nominator/denominator
    
if experiment_name == 'PMG 284 monitoring' or experiment_name == 'PMG 284 monitoring with full mixture spectrum':
    integrals = integrals.apply(pd.to_numeric)
    nominator = np.array(integrals.iloc[:,:3])
    if how_to_divide == 'whole spectrum':
        denominator = np.array(integrals[['whole_spectrum']])
    elif how_to_divide == 'sum':
        denominator = np.array(integrals.iloc[:,:3]).sum(axis=1).reshape(-1,1)
    python_integrals_proportions = nominator/denominator
    
if experiment_name == 'PMG 287 monitoring' or experiment_name == 'PMG 287 monitoring with full mixture spectrum':
    integrals = integrals.apply(pd.to_numeric)
    nominator = np.array(integrals.iloc[:,:3])
    if how_to_divide == 'whole spectrum':
        denominator = np.array(integrals[['whole_spectrum']])
    elif how_to_divide == 'sum':
        denominator = np.array(integrals.iloc[:,:3]).sum(axis=1).reshape(-1,1)
    python_integrals_proportions = nominator/denominator

### Setting time for particular experiments

In [ ]:
time_range = {
                'Saccharose hydrolysis': (1/60)*np.loadtxt(sucrose_time_axis_path)[::10],
                'PMG 284 monitoring': [(1/60)*5*(el-1) for el in list(range(1,1000,10))],
                'PMG 287 monitoring': [(1/60)*5*(el-1) for el in list(range(1,1000,10))],
                'Saccharose hydrolysis with full mixture spectrum': (1/60)*np.loadtxt(sucrose_time_axis_path)[::10],
                'PMG 284 monitoring with full mixture spectrum': [(1/60)*5*(el-1) for el in list(range(1,1000,10))],
                'PMG 287 monitoring with full mixture spectrum': [(1/60)*5*(el-1) for el in list(range(1,1000,10))]
}

time_range_integrals_python = {
                'Saccharose hydrolysis': (1/60)*np.loadtxt(sucrose_time_axis_path),
                'PMG 284 monitoring': [(1/60)*5*(el-1) for el in list(range(1,1000))],
                'PMG 287 monitoring': [(1/60)*5*(el-1) for el in list(range(1,1000))],
                'Saccharose hydrolysis with full mixture spectrum': (1/60)*np.loadtxt(sucrose_time_axis_path),
                'PMG 284 monitoring with full mixture spectrum': [(1/60)*5*(el-1) for el in list(range(1,1000))],
                'PMG 287 monitoring with full mixture spectrum': [(1/60)*5*(el-1) for el in list(range(1,1000))]
}

time_range_integrals_mnova = {
                'Saccharose hydrolysis': (1/60)*np.loadtxt(sucrose_time_axis_path)[:-1],
                'PMG 284 monitoring': [(1/60)*5*(el-1) for el in list(range(1,1001))],
                'PMG 287 monitoring': [(1/60)*5*(el-1) for el in list(range(1,1001))],
                'Saccharose hydrolysis with full mixture spectrum': (1/60)*np.loadtxt(sucrose_time_axis_path)[:-1],
                'PMG 284 monitoring with full mixture spectrum': [(1/60)*5*(el-1) for el in list(range(1,1001))],
                'PMG 287 monitoring with full mixture spectrum': [(1/60)*5*(el-1) for el in list(range(1,1001))]

}

### Visualising results for different parameters, different components

In [ ]:
component_nr = 0

In [ ]:
lin_regr = False

In [ ]:
fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all', sharey='all')
fig.suptitle('Proportion of ' + substances_names[experiment_name][component_nr], fontsize=50)
fig.set_size_inches(30, 18, forward=True)

fig.text(0.5, 0.92, 'Kappa components', ha='center', size=25)
fig.text(0.06, 0.5, 'Kappa mixture', va='center', rotation='vertical', size=25)

for i, ax in enumerate(axs):
    kappa = all_kappas[i]
    for j, axx in enumerate(ax):
        kappa_th = all_kappas_th[j]
        try:
            y = results_dict[(kappa, kappa_th)][:, component_nr]
            axx.plot(time_range[experiment_name], y, 'p', markersize=5, 
                     color=saturated_colors_for_components[experiment_name][component_nr])
            if lin_regr:
                t = np.array(time_range[experiment_name])
                t_and_ones = np.vstack([t, np.ones(len(t))]).T
                slope, intercept = np.linalg.lstsq(t_and_ones, y)[0]
                axx.plot(time_range[experiment_name], slope*t+intercept, linewidth=2.5)
        except TypeError:
            axx.plot(0,0)
        
cols = [str(kappa_th) for kappa_th in all_kappas_th]
rows = [str(kappa) for kappa in all_kappas]       

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size=20)
    ax.set_ylim(0,1)
    
for ax, col in zip(axs[-1], cols):
        ax.set_xlabel('Time [min]', size=10)

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90, size=20)
#fig.savefig(results_paths[experiment_name]+'comparison_for_different_kappas_'+ substances_names[experiment_name][component_nr]+'.png')

### Visualising results for different parameters, all components together

In [ ]:
if experiment_name == 'Saccharose hydrolysis with full mixture spectrum' or experiment_name == 'PMG 284 monitoring with full mixture spectrum':
    fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all')
else:
    fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all', sharey='all')
#fig.suptitle('Proportion of substrates and product', fontsize=50)
fig.set_size_inches(30, 18, forward=True)

ylims_dict_sacch_hydr_full_mix = {0.005: 0.05, 0.01: 0.05, 0.1: 0.21, 0.25: 0.63, 0.5:1.}
ylims_dict_sacch_hydr_full_mix = {0.005: 0.05, 0.01: 0.05, 0.1: 0.21, 0.25: 0.63, 0.5:1.}


fig.text(0.5, 0.92, 'Kappa components', ha='center', size=35)
fig.text(0.06, 0.5, 'Kappa mixture', va='center', rotation='vertical', size=35)

for i, ax in enumerate(axs):
    kappa = all_kappas[i]
    for j, axx in enumerate(ax):
        kappa_th = all_kappas_th[j]
        try:
            y = results_dict[(kappa, kappa_th)]
            for i in range(y.shape[1]):
                if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
                    axx.axis(ymin=0, ymax=ylims_dict_sacch_hydr_full_mix[kappa])
                    axx.plot(time_range[experiment_name], y[:,i], 'p', markersize=5, 
                             label=substances_names[experiment_name][i],
                            color=saturated_colors_for_components[experiment_name][i])
                else:
                    axx.plot(time_range[experiment_name], y[:,i], 'p', markersize=5, 
                             label=substances_names[experiment_name][i],
                            color=saturated_colors_for_components[experiment_name][i])
        except TypeError:
            axx.plot(0,0)
        except AttributeError:
            axx.plot(0,0)
        
cols = [str(kappa_th) for kappa_th in all_kappas_th]
rows = [str(kappa) for kappa in all_kappas]       

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size=30)
    if experiment_name == 'Saccharose hydrolysis with full mixture spectrum' or experiment_name == 'PMG 287 monitoring with full mixture spectrum' or experiment_name == 'PMG 284 monitoring with full mixture spectrum':
        pass
    else:
        ax.set_ylim(0,1)

for ax, col in zip(axs[-1], cols):
    ax.set_xlabel('Time [min]', size=15)

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90, size=30)
handles, labels = ax.get_legend_handles_labels()

if experiment_name == 'Saccharose hydrolysis' or experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
    fig.legend(handles, labels, loc='upper right', prop={'size': 20}, markerscale=5)
else:
    fig.legend(handles, labels, loc='upper right', prop={'size': 25}, markerscale=5)
#fig.savefig(results_paths[experiment_name]+'comparison_for_different_kappas_components_separately.png')

### Visualising results for different parameters, all components together + integrals

In [ ]:
# colors_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
#              '#7f7f7f', '#bcbd22', '#17becf']

ylims_dict_sacch_hydr_full_mix = {0.01: 0.05, 0.1: 0.21, 0.25: 0.63, 0.5:1.}

python_or_mnova = 'Mnova'
if python_or_mnova == 'Mnova':
    integrals_proportions = mnova_integrals_proportions
    time_range_integrals = time_range_integrals_mnova
elif python_or_mnova == 'Python':
    integrals_proportions = python_integrals_proportions
    time_range_integrals = time_range_integrals_python

if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
    fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all')
else:
    fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all', sharey='all')
fig.suptitle('Proportions of substrates and products', fontsize=80)
fig.set_size_inches(40, 25, forward=True)

fig.text(0.5, 0.92, 'Kappa components', ha='center', size=40)
fig.text(0.06, 0.5, 'Kappa mixture', va='center', rotation='vertical', size=40)

for i, ax in enumerate(axs):
    kappa = all_kappas[i]
    for j, axx in enumerate(ax):
        kappa_th = all_kappas_th[j]
        y = results_dict[(kappa, kappa_th)]
        if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
            pass
        else:
            for i in range(integrals_proportions.shape[1]):
                axx.plot(time_range_integrals[experiment_name], integrals_proportions[:,i], 'p', 
                         markersize=5, label=python_or_mnova + ' normalized integral for ' + substances_names[experiment_name][i],
                         color=colors_for_components[experiment_name][i]
                         #alpha=0.1
                        )
        for i in range(y.shape[1]):
            if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
                axx.axis(ymin=0, ymax=ylims_dict_sacch_hydr_full_mix[kappa])
                axx.plot(time_range[experiment_name], y[:,i], 'p', markersize=5, 
                        label=substances_names[experiment_name][i],
                        color=saturated_colors_for_components[experiment_name][i])
            else:
                axx.plot(time_range[experiment_name], y[:,i], 'p', markersize=5, 
                         label=substances_names[experiment_name][i],
                        color=saturated_colors_for_components[experiment_name][i])
            axx.tick_params(axis='both', which='major', labelsize=20)


#         except AttributeError:
#             axx.plot(0,0)
        
cols = [str(kappa_th) for kappa_th in all_kappas_th]
rows = [str(kappa) for kappa in all_kappas]       

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size=40)
    if experiment_name == 'Saccharose hydrolysis with full mixture spectrum' or experiment_name == 'PMG 287 monitoring with full mixture spectrum':
        pass
    else:
        ax.set_ylim(0,1)
    
for ax, col in zip(axs[-1], cols):
    ax.set_xlabel('Time [min]', size=20)

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90, size=40)
handles, labels = ax.get_legend_handles_labels()

legend = fig.legend(handles, labels, loc='upper right', prop={'size': 30}, markerscale=5)

# for lh in legend.legendHandles[i+1:]: 
#     lh.set_alpha(0.5)

# if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
#     fig.savefig(results_paths[experiment_name] +
#                                         'comparison_for_different_kappas_components_separately.png')

# else:
#     fig.savefig(results_paths[experiment_name] +
#                                         'comparison_for_different_kappas_components_separately_plus_integrals_' +
#                                         python_or_mnova +
#                                         '.png')

### Single plot magnetstein + integral (chosen values of parameters)

In [ ]:
# saccharose: anything works, PMG 284: anything works, PMG 287: 0.1, 1
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
# colors_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
#              '#7f7f7f', '#bcbd22', '#17becf']

include_ticks = True

ylims_dict_sacch_hydr_full_mix = {0.01: 0.05, 0.1: 0.21, 0.25: 0.63, 0.5:1.}

python_or_mnova = 'Mnova'
if python_or_mnova == 'Mnova':
    integrals_proportions = mnova_integrals_proportions
    time_range_integrals = time_range_integrals_mnova
elif python_or_mnova == 'Python':
    integrals_proportions = python_integrals_proportions
    time_range_integrals = time_range_integrals_python

include_integrals = True
if include_integrals:
    for i in range(integrals_proportions.shape[1]):
        plt.plot(time_range_integrals[experiment_name], integrals_proportions[:,i], 'p', 
                 markersize=5, label=python_or_mnova + ' integration, ' + substances_names[experiment_name][i],
                 color = colors_for_components[experiment_name][i]
                 #alpha=0.1
                )
for i in range(y.shape[1]):
    if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
        plt.plot(time_range[experiment_name], y[:, i], 'p', 
                 label = 'Magnetstein, ' + substances_names[experiment_name][i],
                color=saturated_colors_for_components[experiment_name][i])
        plt.ylim(0, ylims_dict_sacch_hydr_full_mix[best_kappa])
    else:
        plt.plot(time_range[experiment_name], y[:, i], 'p', 
                 label = 'Magnetstein, ' + substances_names[experiment_name][i],
                color=saturated_colors_for_components[experiment_name][i])
        if experiment_name == 'PMG 287 monitoring with full mixture spectrum':
            plt.ylim(0, 0.4)
        else:
            plt.ylim(0,1)

if include_ticks:
    plt.xlabel('Time [min]', size=15)
    plt.ylabel('Proportion', size=15)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
else:
    plt.xlabel('Time', size=15)
    plt.xticks([])
    plt.yticks([])
    
leg = plt.legend(title='Method, substance')    
plt.gcf().set_size_inches(10, 5)

# for lh in leg.legendHandles[:y.shape[1]]: 
#     lh.set_alpha(0.4)

# if include_ticks:
#     plt.savefig(results_paths[experiment_name]+'all_components_on_one_plot_plus_integrals_'+
#                                                         python_or_mnova +
#                                                         '_kappa_' +
#                                                     str(best_kappa)+'_kappa_th_' + 
#                                                     str(best_kappa_th) +'.png', dpi=300)
# else:
#     plt.savefig(results_paths[experiment_name]+'all_components_on_one_plot_plus_integrals_'+
#                                                         python_or_mnova +
#                                                         '_kappa_' +
#                                                     str(best_kappa)+'_kappa_th_' + 
#                                                     str(best_kappa_th) +'_no_labels.png', dpi=300)

### Single plot for magnetstein (for chosen values of parameters)

In [ ]:
# saccharose, PMG 284, PMG 287: anything works
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
# colors_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
#              '#7f7f7f', '#bcbd22', '#17becf']

for i in range(y.shape[1]):
    if experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
        plt.plot(time_range[experiment_name], y[:, i], 'p', 
                 label = 'Magnetstein, ' + substances_names[experiment_name][i],
                color=saturated_colors_for_components[experiment_name][i])
    else:
        plt.plot(time_range[experiment_name], y[:, i], 'p', 
                 label = 'Magnetstein, ' + substances_names[experiment_name][i],
                color=saturated_colors_for_components[experiment_name][i])

plt.xlabel('Time', size=15)
plt.ylabel('Relative amount', size=15)
# plt.xticks([])
# plt.yticks([])
 
plt.gcf().set_size_inches(10, 5)

# for lh in leg.legendHandles[:y.shape[1]]: 
#     lh.set_alpha(0.4)

# plt.savefig(results_paths[experiment_name]+'all_components_on_one_plot_magnetstein_only'+
#                                                     '_kappa_' +
#                                                 str(best_kappa)+'_kappa_th_' + 
#                                                 str(best_kappa_th) +'.png', dpi=300)

### Single plot magnetstein + MNova + Python (chosen values of parameters)

In [ ]:
# saccharose: anything works, PMG 284: anything works, PMG 287: 0.1, 1
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
#%matplotlib notebook

In [ ]:
include_integrals = True
if include_integrals:
    for i in range(mnova_integrals_proportions.shape[1]):
        plt.plot(time_range_integrals_mnova[experiment_name], mnova_integrals_proportions[:,i], 'p', 
                 markersize=5, label='MNova integration, ' + substances_names[experiment_name][i],
                 color = colors_for_components[experiment_name][i]
                 #alpha=0.1
                )
    for i in range(python_integrals_proportions.shape[1]):
        plt.plot(time_range_integrals_python[experiment_name], python_integrals_proportions[:,i], 'v', 
                 markersize=5, label='Python integration, ' + substances_names[experiment_name][i],
                 color = colors_for_components[experiment_name][i]
                 #alpha=0.1
                )
for i in range(y.shape[1]):
    plt.plot(time_range[experiment_name], y[:, i], 'p', 
             label = 'Magnetstein, ' + substances_names[experiment_name][i],
            color=saturated_colors_for_components[experiment_name][i])

    plt.title('Proportions in time')
    
plt.xlabel('Time [min]')
    
if experiment_name == 'PMG 287 monitoring with full mixture spectrum':
    plt.ylim(0, 0.4)
else:
    plt.ylim(0,1)
    
leg = plt.legend(title='Method, substance')    
# for lh in leg.legendHandles[:y.shape[1]]: 
#     lh.set_alpha(0.4)
plt.gcf().set_size_inches(10, 5)
# plt.savefig(results_paths[experiment_name]+'all_components_on_one_plot_plus_integrals_MNova_Python'+
#                                                     '_kappa_' +
#                                                 str(best_kappa)+'_kappa_th_' + 
#                                                 str(best_kappa_th) +'.png')

### Integral only

In [ ]:
# colors_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
#              '#7f7f7f', '#bcbd22', '#17becf']

python_or_mnova = 'Mnova'
if python_or_mnova == 'Mnova':
    integrals_proportions = mnova_integrals_proportions
    time_range_integrals = time_range_integrals_mnova
elif python_or_mnova == 'Python':
    integrals_proportions = python_integrals_proportions
    time_range_integrals = time_range_integrals_python

    
for i in range(integrals_proportions.shape[1]):
    plt.plot(time_range_integrals[experiment_name], integrals_proportions[:,i], 'p', 
             markersize=5, label=python_or_mnova + ' normalized integral for ' + substances_names[experiment_name][i],
             color = colors_for_components[experiment_name][i]#, alpha=0.03
            )

plt.title('Proportions in time')

plt.xlabel('Time [min]')
    
#plt.ylim(0,1)
    
leg = plt.legend()    

plt.gcf().set_size_inches(10, 5)
# plt.savefig(results_paths[experiment_name]+'all_components_integrals_on_one_plot_plus_integrals_' +
#                                                     python_or_mnova +
#                                                     '_kappa' +
#                                                 str(best_kappa)+'_kappa_th_' + 
#                                                 str(best_kappa_th) +'.png')

### Visualising results for different parameters, all components added + integrals

In [ ]:
components_numbers = [0]
product_numbers = [1,2]

python_or_mnova = 'Mnova'
if python_or_mnova == 'Mnova':
    integrals_proportions = mnova_integrals_proportions
    time_range_integrals = time_range_integrals_mnova
elif python_or_mnova == 'Python':
    integrals_proportions = python_integrals_proportions
    time_range_integrals = time_range_integrals_python

colors_list = ['grey', 'purple']

fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all', sharey='all')
fig.suptitle('Proportion of substrates and product', fontsize=50)
fig.set_size_inches(40, 25, forward=True)

fig.text(0.5, 0.92, 'Kappa components', ha='center', size=25)
fig.text(0.06, 0.5, 'Kappa mixture', va='center', rotation='vertical', size=25)

for i, ax in enumerate(axs):
    kappa = all_kappas[i]
    for j, axx in enumerate(ax):
        kappa_th = all_kappas_th[j]

        #components
        y = 0
        for nr in components_numbers:
            y = y + results_dict[(kappa, kappa_th)][:, nr]
        axx.plot(time_range[experiment_name], y, 'p', markersize=5,
                label='substrates',
                    color = colors_list[0])
        #product
        y = 0
        for nr in product_numbers:
            y = y + results_dict[(kappa, kappa_th)][:, nr]
        axx.plot(time_range[experiment_name], y, 'p', markersize=5,
                label='product',
                    color = colors_list[1])
        y = results_dict[(kappa, kappa_th)]
        axx.plot(time_range_integrals[experiment_name], integrals_proportions[:,components_numbers].sum(axis=1), 'p', 
                     markersize=5, label=python_or_mnova + ' normalized integral for substrates',
                     color = colors_list[0], alpha=0.03
                    )
        axx.plot(time_range_integrals[experiment_name], integrals_proportions[:,product_numbers].sum(axis=1), 'p', 
                     markersize=5, label=python_or_mnova + ' normalized integral for products',
                     color = colors_list[1], alpha=0.03
                    )
        
cols = [str(kappa_th) for kappa_th in all_kappas_th]
rows = [str(kappa) for kappa in all_kappas]       

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size=20)
    ax.set_ylim(0,1)
    
for ax, col in zip(axs[-1], cols):
    ax.set_xlabel('Time [min]', size=10)

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90, size=20)
handles, labels = ax.get_legend_handles_labels()

legend = fig.legend(handles, labels, loc='upper right', prop={'size': 25}, markerscale=5)

#fig.savefig(results_paths[experiment_name]+
#                 'comparison_for_different_kappas_components_together_plus_integrals_' +
#                 python_or_mnova + 
#                 '.png')

### Components together

In [ ]:
components_numbers = [0]
lin_regr = False

In [ ]:
names = [substances_names[experiment_name][nr] for nr in components_numbers]

fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all', sharey='all')
fig.suptitle('Added proportion of components: ' + str(names), fontsize=50)
fig.set_size_inches(30, 18, forward=True)

fig.text(0.5, 0.92, 'Kappa components', ha='center', size=25)
fig.text(0.06, 0.5, 'Kappa mixture', va='center', rotation='vertical', size=25)

for i, ax in enumerate(axs):
    kappa = all_kappas[i]
    for j, axx in enumerate(ax):
        kappa_th = all_kappas_th[j]
        try:
            y = 0
            for nr in components_numbers:
                y = y + results_dict[(kappa, kappa_th)][:, nr]
            axx.plot(time_range[experiment_name], y, 'p', markersize=5,
                    color='grey')

            if lin_regr:
                t = np.array(time_range[experiment_name])
                t_and_ones = np.vstack([t, np.ones(len(t))]).T
                slope, intercept = np.linalg.lstsq(t_and_ones, y)[0]
                slope, intercept = np.linalg.lstsq(t_and_ones, y)[0]
                axx.plot(time_range[experiment_name], slope*t+intercept, linewidth=2.5)
        except TypeError:
            axx.plot(0,0)
        
cols = [str(kappa_th) for kappa_th in all_kappas_th]
rows = [str(kappa) for kappa in all_kappas]       

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size=20)
    ax.set_ylim(0,1)
    
for ax, col in zip(axs[-1], cols):
    ax.set_xlabel('Time [min]', size=10)

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90, size=20)
#fig.savefig(results_paths[experiment_name]+'comparison_for_different_kappas_sum_of_components_' +
#                                                   str(names) +'.png')

### Single plot for chosen values of parameters

#### Components separately

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
component_nr = 2
lin_regr = False

In [ ]:
plt.plot(time_range[experiment_name], y[:, component_nr], 'p',
        color = saturated_colors_for_components[experiment_name][component_nr])
if lin_regr:
    t = np.array(time_range[experiment_name])
    t_and_ones = np.vstack([t, np.ones(len(t))]).T
    slope, intercept = np.linalg.lstsq(t_and_ones, y[:, component_nr])[0]
    plt.plot(time_range[experiment_name], slope*t+intercept)
#plt.title('Proportion of '+substances_names[experiment_name][component_nr])
plt.title('Proportion of ' + substances_names[experiment_name][component_nr])

for ax, col in zip(axs[-1], cols):
    plt.xlabel('Time [min]')

#### Components added together

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
component_numbers = [0]
lin_regr = False

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
plt.plot(time_range[experiment_name], y[:,component_numbers].sum(1), 'p',
        color='grey')
if lin_regr:
    t = np.array(time_range[experiment_name])
    t_and_ones = np.vstack([t, np.ones(len(t))]).T
    slope, intercept = np.linalg.lstsq(t_and_ones, y[:, component_nr])[0]
    plt.plot(time_range[experiment_name], slope*t+intercept)
plt.title('Proportion of components: ' + str(component_numbers))

for ax, col in zip(axs[-1], cols):
    plt.xlabel('Time [min]')
#plt.ylim(0.5,1)

#### Noise proportion

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
plt.plot(time_range[experiment_name], 1-y.sum(axis=1), 'p',
        color='grey')
plt.title('Proportion of noise in mixture')

for ax, col in zip(axs[-1], cols):
    plt.xlabel('Time [min]')

Another way:

In [ ]:
with open(results_paths[experiment_name]+'noise_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = [sum(l) for l in y]

In [ ]:
plt.plot(time_range[experiment_name], y, 'p',
        color='grey')
plt.title('Proportion of noise in mixture')
for ax, col in zip(axs[-1], cols):
    plt.xlabel('Time [min]')

#### Product and substrates together

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
component_numbers = [0]
product_numbers = [1, 2]
lin_regr = False

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
#Version when we don't have spectrum of product (i.e. we assume that noise in mixture is product)

colors_list = ['grey', 'purple']

python_or_mnova = 'Python'
if python_or_mnova == 'Mnova':
    integrals_proportions = mnova_integrals_proportions
    time_range_integrals = time_range_integrals_mnova
elif python_or_mnova == 'Python':
    integrals_proportions = python_integrals_proportions
    time_range_integrals = time_range_integrals_python

include_integrals = True
if include_integrals:
    plt.plot(time_range_integrals[experiment_name], integrals_proportions[:,component_numbers].sum(axis=1), 'p', 
                 markersize=5, label=python_or_mnova + ' normalized integral for substartes',
                 color = colors_list[0], alpha=0.03
                )
    plt.plot(time_range_integrals[experiment_name], integrals_proportions[:,product_numbers].sum(axis=1), 'p', 
                 markersize=5, label=python_or_mnova + ' normalized integral for products',
                 color = colors_list[1], alpha=0.03
                )

plt.plot(time_range[experiment_name], y[:,component_numbers].sum(1), 'p', label='Substrates', color = colors_list[0])
plt.plot(time_range[experiment_name], y[:,product_numbers].sum(1), 'p', label='Product', color = colors_list[1])

plt.ylim(0,1)
#plt.plot(list(range(y.shape[0]*10))[::10], 1-y.sum(axis=1), 'p', label='Product')

#leg = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))    
leg = plt.legend(loc='upper left', prop = {"size": 7})    

plt.gcf().set_size_inches(10, 5)

plt.title('Proportion of substrates/product in time')

for ax, col in zip(axs[-1], cols):
    plt.xlabel('Time [min]')
        
#plt.savefig(results_paths[experiment_name]+
#                             'substrates_and_product_together_plus_integrals_' + 
#                             python_or_mnova + 
#                             '_kappa_' + 
#                             str(best_kappa) +
#                             '_kappa_th_' +
#                             str(best_kappa_th)
#                             +'.png')

In [ ]:
#Version when we have spectrum of product

colors_list = ['grey', 'purple']

#Below: version with rescaled time (every 10th measurement was checked)
plt.plot(time_range[experiment_name], y[:,component_numbers].sum(1), 'p', label='Substrates',
        color=colors_list[0])
#plt.ylim(0.5,1)
plt.plot(time_range[experiment_name], y[:,product_numbers].sum(1), 'p', label='Product',
        color=colors_list[1])
plt.title('Proportion of substrates/product in time')

for ax, col in zip(axs[-1], cols):
    plt.xlabel('Time [min]')
        
plt.legend()
# plt.savefig(results_paths[experiment_name]+'substrates_and_product_together_' + str(best_kappa) +'_'+str(best_kappa_th)+'.png')

### Noise in components

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'noise_proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
plt.plot(time_range[experiment_name], y, 'p', 
        color='grey')
plt.title('Proportion of noise in components')

for ax, col in zip(axs[-1], cols):
    plt.xlabel('Time [min]')

### Noise proportion

In [ ]:
components_numbers = list(range(len(substances_names[experiment_name])))
lin_regr = False

In [ ]:
names = [substances_names[experiment_name][nr] for nr in components_numbers]

fig, axs = plt.subplots(len(all_kappas), len(all_kappas_th), sharex='all', sharey='all')
fig.suptitle('Proportion of signal removed from mixture', fontsize=50)
fig.set_size_inches(30, 18, forward=True)

fig.text(0.5, 0.92, 'Kappa components', ha='center', size=25)
fig.text(0.06, 0.5, 'Kappa mixture', va='center', rotation='vertical', size=25)

for i, ax in enumerate(axs):
    kappa = all_kappas[i]
    for j, axx in enumerate(ax):
        kappa_th = all_kappas_th[j]
        try:
            y = 1
            for nr in components_numbers:
                y = y - results_dict[(kappa, kappa_th)][:, nr]
            axx.plot(time_range[experiment_name], y, 'p', markersize=5,
                    color='grey')
            if lin_regr:
                t = np.array(time_range[experiment_name])
                t_and_ones = np.vstack([t, np.ones(len(t))]).T
                slope, intercept = np.linalg.lstsq(t_and_ones, y)[0]
                slope, intercept = np.linalg.lstsq(t_and_ones, y)[0]
                axx.plot(time_range[experiment_name], slope*t+intercept, linewidth=2.5)
        except TypeError:
            axx.plot(0,0)
        
cols = [str(kappa_th) for kappa_th in all_kappas_th]
rows = [str(kappa) for kappa in all_kappas]       

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size=20)
    ax.set_ylim(0,1)
    
for ax, col in zip(axs[-1], cols):
    ax.set_xlabel('Time [min]', size=10)

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90, size=20)
#fig.savefig(results_paths[experiment_name]+'proportion_of_removed_signal.png')

### Visualisation of removed signal in mixture

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'noise_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
with open(results_paths[experiment_name]+'common_horizontal_axis_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    cha = pickle.load(f)
    cha = np.array(cha)

In [ ]:
moment_of_time = 1

In [ ]:
noise = y[moment_of_time-1,:]

In [ ]:
mixture_time_data = pd.read_csv(mixture_paths[experiment_name], sep = mixture_separators[experiment_name])
if experiment_name == 'Saccharose hydrolysis' or experiment_name == 'Saccharose hydrolysis with full mixture spectrum':
    ppm = mixture_time_data.iloc[:,:-1].iloc[:,0:1]
    every_10th = mixture_time_data.iloc[:,:-1].iloc[:,1:].iloc[:,::10]
    mixture_time_data = pd.concat((ppm, every_10th), axis=1)
elif experiment_name == 'PMG 284 monitoring' or experiment_name == 'PMG 284 monitoring with full mixture spectrum':
    ppm = mixture_time_data.iloc[:,:-1].iloc[:,0:1]
    every_10th = mixture_time_data.iloc[:,:-1].iloc[:,1:].iloc[:,::10]
    mixture_time_data = pd.concat((ppm, every_10th), axis=1)
elif experiment_name == 'PMG 287 monitoring' or experiment_name == 'PMG 287 monitoring with full mixture spectrum':
    ppm = mixture_time_data.iloc[:,:-1].iloc[:,0:1]
    every_10th = mixture_time_data.iloc[:,:-1].iloc[:,1:].iloc[:,::10]
    mixture_time_data = pd.concat((ppm, every_10th), axis=1)

In [ ]:
if experiment_name == 'PMG 284 monitoring' or experiment_name == 'PMG 284 monitoring with full mixture spectrum':
    mixture_time_data.fillna(0, inplace=True)

In [ ]:
names = ['ppm'] + ['t' + str(nb) for nb in range(1, mixture_time_data.shape[1])]

In [ ]:
mixture_time_data.columns = names

In [ ]:
def load_spectrum(mixture_time_data, moment_of_time):
    ppm = mixture_time_data['ppm']
    intensity = mixture_time_data['t'+str(moment_of_time)]
    sp = NMRSpectrum(confs = list(zip(ppm, intensity)))
    return sp

In [ ]:
mix = load_spectrum(mixture_time_data, moment_of_time)
mix.normalize()

In [ ]:
%matplotlib widget

In [ ]:
bar_width = np.mean([cha[i]-cha[i-1] for i in range(1, len(cha))])

plt.gca().invert_xaxis()
NMRSpectrum.plot_all([mix], profile=True)
plt.bar(cha, noise, alpha=0.7, label='signal removed from mixture', width=bar_width)
plt.legend()
plt.savefig(results_paths[experiment_name] + 'noise_removed_from_mixture.png')

### Visualisation of removed signal from mixture + spectra of products

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
with open(results_paths[experiment_name]+'noise_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
with open(results_paths[experiment_name]+'common_horizontal_axis_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    cha = pickle.load(f)
    cha = np.array(cha)

In [ ]:
moment_of_time = 1

In [ ]:
noise = y[moment_of_time-1,:]

In [ ]:
reagents = []
for i, path in enumerate(reagents_paths[experiment_name]):
    reag = pd.read_csv(path, sep = reagents_separators[experiment_name])
    if experiment_name == 'PMG 284 monitoring' and i==2:
        reag.fillna(0, inplace=True)
    sp = NMRSpectrum(confs = list(zip(reag.iloc[:,0], reag.iloc[:,1])))
    sp.normalize()
    reagents.append(sp)

In [ ]:
if experiment_name == 'Saccharose hydrolysis':
    products = reagents[-2:]
else:
    products = reagents[-1:]

In [ ]:
bar_width = np.mean([cha[i]-cha[i-1] for i in range(1, len(cha))])

plt.gca().invert_xaxis()
NMRSpectrum.plot_all(products, profile=True)
plt.bar(cha, noise, alpha=0.7, label='noise', width=bar_width)
plt.legend()

### Visualisation of removed signal in components

This plot will not work for saccharose hydrolysis, because the common mass axis is not saved for this experiment (because it's different for every time point due to baseline correction).

In [ ]:
best_kappa = 0.5
best_kappa_th = 0.5

In [ ]:
moment_of_time = 60

In [ ]:
components_nrs = [0]

In [ ]:
with open(results_paths[experiment_name]+'proportions_in_times_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)
    current_proportions = y[moment_of_time-1, components_nrs]

In [ ]:
with open(results_paths[experiment_name]+'noise_in_components_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)

In [ ]:
with open(results_paths[experiment_name]+'common_horizontal_axis_'+str(best_kappa)+'_'+str(best_kappa_th)+'.pkl', 'rb') as f:
    cha = pickle.load(f)
    cha = np.array(cha)

In [ ]:
noise_in_components = y[moment_of_time-1,:]

In [ ]:
reagents = []
for i, path in enumerate(reagents_paths[experiment_name]):
    reag = pd.read_csv(path, sep = reagents_separators[experiment_name])
    if experiment_name == 'PMG 284 monitoring' and i==2:
        reag.fillna(0, inplace=True)
    sp = NMRSpectrum(confs = list(zip(reag.iloc[:,0], reag.iloc[:,1])))
    sp.normalize()
    reagents.append(sp)

In [ ]:
%matplotlib notebook

In [ ]:
bar_width = np.mean([cha[i]-cha[i-1] for i in range(1, len(cha))])

plt.gca().invert_xaxis()
for i in components_nrs:
    reagents[i].plot(profile=True,
                    color=saturated_colors_for_components[experiment_name][i])
plt.bar(cha, noise_in_components, alpha=0.7, label='noise in components', width=bar_width, color='grey')
plt.legend()

### Mixture and components

In [ ]:
reagents = []
for i, path in enumerate(reagents_paths[experiment_name]):
    reag = pd.read_csv(path, sep = reagents_separators[experiment_name])
    sp = NMRSpectrum(confs = list(zip(reag.iloc[:,0], reag.iloc[:,1])))
    sp.normalize()
    reagents.append(sp)

In [ ]:
mixture_time_data = pd.read_csv(mixture_paths[experiment_name], sep = mixture_separators[experiment_name])
if experiment_name == 'Saccharose hydrolysis':
    ppm = mixture_time_data.iloc[:,:-1].iloc[:,0:1]
    every_10th = mixture_time_data.iloc[:,:-1].iloc[:,1:].iloc[:,::10]
    mixture_time_data = pd.concat((ppm, every_10th), axis=1)
elif experiment_name == 'PMG 284 monitoring':
    ppm = mixture_time_data.iloc[:,:-1].iloc[:,0:1]
    every_10th = mixture_time_data.iloc[:,:-1].iloc[:,1:].iloc[:,::10]
    mixture_time_data = pd.concat((ppm, every_10th), axis=1)
elif experiment_name == 'PMG 287 monitoring':
    ppm = mixture_time_data.iloc[:,:-1].iloc[:,0:1]
    every_10th = mixture_time_data.iloc[:,:-1].iloc[:,1:].iloc[:,::10]
    mixture_time_data = pd.concat((ppm, every_10th), axis=1)

In [ ]:
if experiment_name == 'PMG 284 monitoring':
    mixture_time_data.fillna(0, inplace=True)

In [ ]:
names = ['ppm'] + ['t' + str(nb) for nb in range(1, mixture_time_data.shape[1])]

In [ ]:
mixture_time_data.columns = names

In [ ]:
def load_spectrum(mixture_time_data, moment_of_time):
    ppm = mixture_time_data['ppm']
    intensity = mixture_time_data['t'+str(moment_of_time)]
    sp = NMRSpectrum(confs = list(zip(ppm, intensity)))
    sp.normalize()
    return sp

In [ ]:
plt.gca().invert_xaxis()
NMRSpectrum.plot_all(reagents + [load_spectrum(mixture_time_data, 100)], profile=True)

In [ ]:
plt.gca().invert_xaxis()
NMRSpectrum.plot_all(reagents + [load_spectrum(mixture_time_data, 1)], profile=True)